K. Collins

> Exercise: Project 1 Part 5

> Week: 5

> Task:  Use linear regression to create a model to predict and analyze sales


In [ ]:
!pip install category_encoders
import pandas as pd

#import data into dataframe
filename = '/content/drive/MyDrive/CodingDojo/Week_2/PandasForDataManipulation/sales_predictions.csv'
df = pd.read_csv(filename)

#inspect data
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


(8523, 12)

In [ ]:
# Data inspection of finds the following:
#  Finding 1) There are blanks in the 'Item Weight' column - I observe association between "Item_Identifier" and "Item_Weight".
#     Solution - Each 'Item_Identifier' has a unqiue 'Item_Weight'. Use mode() to associate correct weight per item identifier.
#  Finding 2) There are abbreviations in "Item_Fat_Content" column. There are two unique enumerations; 'Regular' and 'Low Fat'.
#     Solution - Instances of 'reg' and 'LF' are removed and replaced with 'Regular' and 'Low Fat', respectively.   
#  Finding 3) There are zero values in the 'Item_Visibility' column. I Observe assocation between "Item_Identifier" and "Item_Visibility".
#     Solution - Use Mean() on each instance of "Item_Identifier" and replace zero value with mean visibility for that identifier 
#  Finding 4) - There are blanks in the "Outlet_Size" column. I observe association between "Outlet_Size", and the "Outlet_Type" 
#               and "Outlet_Location_Type" columns. Using "Outlet_Type" as the key, I find the following:
#               4.1 (Outlet_Type = Grocery)(Outlet_Location_Type = ALL): Blanks exist. (Outlet Size = Small) for all other values. 
#               4.2 (Outlet_Type = Supermarket Type1)(Outlet_Location_Type = Tier 1): No blanks.
#               4.3 (Outlet_Type = Supermarket Type1)(Outlet_Location_Type = Tier 2): Blanks exist. (Outlet_Size = Small) for all other values. 
#               4.4 (Outlet_Type = Supermarket Type1)(Outlet_Location_Type = Tier 3): No blanks.            
#               4.5 (Outlet_Type = Supermarket Type2)(Outlet_Location_Type = ALL): No blanks.   
#               4.6 (Outlet_Type = Supermarket Type3)(Outlet_Location_Type = ALL): No blanks. 
#               Solution: Use mode() function to replace blanks in 4.1 and 4.3 with 'Small' given Outlet_Size = Small for all other associated 
#                         values. Ommision appears completely random. 
#
# Code sections follow to address each finding  


# Sort by "Item_Identifier". 
df = df.sort_values(by = 'Item_Identifier')
df = df.reset_index(drop=True)

#print initial dataframe
df


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,DRA12,11.60,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,NaN,Tier 3,Grocery Store,283.6308
1,DRA12,11.60,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,NaN,Tier 2,Supermarket Type1,2552.6772
2,DRA12,11.60,LF,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078
3,DRA12,11.60,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,NaN,Tier 2,Supermarket Type1,3829.0158
4,DRA12,11.60,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,NCZ54,14.65,Low Fat,0.083489,Household,161.3552,OUT049,1999,Medium,Tier 1,Supermarket Type1,7148.0288
8519,NCZ54,14.65,Low Fat,0.000000,Household,161.5552,OUT010,1998,NaN,Tier 3,Grocery Store,324.9104
8520,NCZ54,NaN,Low Fat,0.082956,Household,164.0552,OUT027,1985,Medium,Tier 3,Supermarket Type3,5523.4768
8521,NCZ54,NaN,Low Fat,0.145952,Household,160.9552,OUT019,1985,Small,Tier 1,Grocery Store,162.4552


In [ ]:
# Finding 1) There are blanks in the 'Item Weight' column
# Finding 3) There are zero values in the 'Item_Visibility' column. 

# Find the unique weight for each item identifier
setModeWeight = df.groupby(['Item_Identifier'])['Item_Weight'].mean()

# Find the mean item visibility for each item identifier
setMeanVisibility = df.groupby(['Item_Identifier'])['Item_Visibility'].mean()

# temporarily set the dataframe index to the item identifier for key association
df = df.set_index(['Item_Identifier'])

# Filter for cases where item visibility is 0
visibiltyFilter = df.Item_Visibility == 0.0

# Filter for cases where item weight is blank
weightFilter = df.Item_Weight.isna() == True

# replace 0s with the calculated mean of item visibilites for each unique item identifier 
df.loc[visibiltyFilter, 'Item_Visibility'] = setMeanVisibility

# replace blanks with the unique weight for each item identifier
df.loc[weightFilter, 'Item_Weight'] = setModeWeight

# restore to the intial index
df = df.reset_index()

In [ ]:
#Finding 2) There are abbreviations in "Item_Fat_Content" column.

# Identify the enumerations in the initial data
df['Item_Fat_Content'].unique()

array(['Low Fat', 'LF', 'Regular', 'reg', 'low fat'], dtype=object)

In [ ]:
#Finding 2) - Con't. 

# Replace abbreviations with unique 'Item_Fat_Content' enumurations found in the data. 
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace(['LF','low fat','reg'],['Low Fat','Low Fat','Regular'])

# Show enumerations in the modified data
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [ ]:
#Finding 4) - There are blanks in the "Outlet_Size" column.

# Find unique outlet sizes for each outlet type. See the finding description above.
setModeSize = df.groupby(['Outlet_Type'])['Outlet_Size'].agg(pd.Series.mode)

# temporarily set the dataframe index to the outlet type for key association
df = df.set_index(['Outlet_Type'])

# Filter for cases where the outlet size is blank
sizeFilter = df.Outlet_Size.isna() == True

# replace blanks with the outlet size associated with outlet types
df.loc[sizeFilter, 'Outlet_Size'] = setModeSize

# restore to the intial index
df = df.reset_index()

#print dataframe
df

,Outlet_Type,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales
0,Grocery Store,DRA12,11.60,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,Small,Tier 3,283.6308
1,Supermarket Type1,DRA12,11.60,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,Small,Tier 2,2552.6772
2,Supermarket Type1,DRA12,11.60,Low Fat,0.031956,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,992.7078
3,Supermarket Type1,DRA12,11.60,Low Fat,0.031956,Soft Drinks,141.6154,OUT045,2002,Small,Tier 2,3829.0158
4,Supermarket Type1,DRA12,11.60,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,2552.6772
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,Supermarket Type1,NCZ54,14.65,Low Fat,0.083489,Household,161.3552,OUT049,1999,Medium,Tier 1,7148.0288
8519,Grocery Store,NCZ54,14.65,Low Fat,0.080426,Household,161.5552,OUT010,1998,Small,Tier 3,324.9104
8520,Supermarket Type3,NCZ54,14.65,Low Fat,0.082956,Household,164.0552,OUT027,1985,Medium,Tier 3,5523.4768
8521,Grocery Store,NCZ54,14.65,Low Fat,0.145952,Household,160.9552,OUT019,1985,Small,Tier 1,162.4552


In [ ]:
df.isnull().sum()
# Null check shows four occurences remaining in the 'Item Weight' feature. Each of these were related to a single instance of the 'Item Identifier', therefore mean() was
# not effective. 

Outlet_Type                  0
Item_Identifier              0
Item_Weight                  4
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Item_Outlet_Sales            0
dtype: int64

In [ ]:
df.corr()
# Check for correlations other than 'Item Identifier' to determine 'Item Weight'. Results show poor correlations to 'Item Weight. Alternative action in next cell. 

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Item_Weight,1.000000,-0.017872,0.025975,-0.013426,0.013168
Item_Visibility,-0.017872,1.000000,-0.004525,-0.075175,-0.128449
Item_MRP,0.025975,-0.004525,1.000000,0.005020,0.567574
Outlet_Establishment_Year,-0.013426,-0.075175,0.005020,1.000000,-0.049135
Item_Outlet_Sales,0.013168,-0.128449,0.567574,-0.049135,1.000000


In [ ]:
# Deletion is not desirable. Decide instead to fill the four values with the median() of the 'Item Weight' feature.
df.Item_Weight.fillna(df.Item_Weight.median(), inplace=True)
df.isnull().sum()

Outlet_Type                  0
Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Item_Outlet_Sales            0
dtype: int64

In [ ]:
# Restore the data's initial column structure
df = df[['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Item_MRP', 
        'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 
        'Item_Outlet_Sales']]

df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,DRA12,11.6,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,Small,Tier 3,Grocery Store,283.6308
1,DRA12,11.6,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,Small,Tier 2,Supermarket Type1,2552.6772
2,DRA12,11.6,Low Fat,0.031956,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078
3,DRA12,11.6,Low Fat,0.031956,Soft Drinks,141.6154,OUT045,2002,Small,Tier 2,Supermarket Type1,3829.0158
4,DRA12,11.6,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772


In [ ]:
import category_encoders as ce

# Create the features matrix
X = df.loc[:, :'Outlet_Type']

# Create the target vector
y = df.loc[:,'Item_Outlet_Sales']

# Create an instance of OHE to convert categorical features to nominal numerics for model consumption
ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)

# Standardize the the features matrix
X = ohe.fit_transform(X)

X.head()

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Item_Identifier_DRA12,Item_Identifier_DRA24,Item_Identifier_DRA59,Item_Identifier_DRB01,Item_Identifier_DRB13,Item_Identifier_DRB24,Item_Identifier_DRB25,Item_Identifier_DRB48,Item_Identifier_DRC01,Item_Identifier_DRC12,Item_Identifier_DRC13,Item_Identifier_DRC24,Item_Identifier_DRC25,Item_Identifier_DRC27,Item_Identifier_DRC36,Item_Identifier_DRC49,Item_Identifier_DRD01,Item_Identifier_DRD12,Item_Identifier_DRD13,Item_Identifier_DRD15,Item_Identifier_DRD24,Item_Identifier_DRD25,Item_Identifier_DRD27,Item_Identifier_DRD37,Item_Identifier_DRD49,Item_Identifier_DRD60,Item_Identifier_DRE01,Item_Identifier_DRE03,Item_Identifier_DRE12,Item_Identifier_DRE13,Item_Identifier_DRE15,Item_Identifier_DRE25,Item_Identifier_DRE27,Item_Identifier_DRE37,Item_Identifier_DRE48,Item_Identifier_DRE49,Item_Identifier_DRE60,Item_Identifier_DRF01,Item_Identifier_DRF03,Item_Identifier_DRF13,...,Item_Fat_Content_Regular,Item_Visibility,Item_Type_Soft Drinks,Item_Type_Dairy,Item_Type_Hard Drinks,Item_Type_Canned,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Snack Foods,Item_Type_Baking Goods,Item_Type_Starchy Foods,Item_Type_Meat,Item_Type_Seafood,Item_Type_Breakfast,Item_Type_Breads,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Others,Item_MRP,Outlet_Identifier_OUT010,Outlet_Identifier_OUT017,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT013,Outlet_Identifier_OUT018,Outlet_Identifier_OUT049,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT046,Outlet_Establishment_Year,Outlet_Size_Small,Outlet_Size_High,Outlet_Size_Medium,Outlet_Location_Type_Tier 3,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 1,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0.068535,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,143.0154,1,0,0,0,0,0,0,0,0,0,1998,1,0,0,1,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0.041178,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,140.3154,0,1,0,0,0,0,0,0,0,0,2007,1,0,0,0,1,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0.031956,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,141.9154,0,0,1,0,0,0,0,0,0,0,2004,1,0,0,0,1,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0.031956,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,141.6154,0,0,0,1,0,0,0,0,0,0,2002,1,0,0,0,1,0,0,1,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0.040912,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,142.3154,0,0,0,0,1,0,0,0,0,0,1987,0,1,0,1,0,0,0,1,0,0


In [ ]:
from sklearn.linear_model import LinearRegression

# Instantiate the linear regression model
LR = LinearRegression(fit_intercept=True)

# fit the features matrix and the target vector
LR.fit(X,y)

# Show sample predictions of the first 20 values
LR.predict(X[0:20])

array([ 210.328125  , 2313.72460938, 2338.46289062, 2121.48632812,
       2213.1953125 , 1863.78515625, 2668.8984375 , 2712.83203125,
        785.1640625 , 4059.8984375 , 2789.68164062,  666.328125  ,
       2769.20507812, 2890.23242188, 2805.19921875, 2496.48046875,
       2846.14257812,  814.56640625,  929.91015625, 4237.44921875])

In [ ]:
# Calculate the coefficient of determination to quantify the model's performance 
score = LR.score(X, y)

# Print the results
print('The linear regression score is {:.2f}'.format(score))

The linear regression score is 0.62


In [ ]:
# Calculate the model's root mean squared error (RMSE)

predictedSales = LR.predict(X)
actualSales = y

# Calculate the sum of squared differences (errors) between sales and predicted sales  
sum = 0
for i in range (0, actualSales.size):
    sum = sum + pow((actualSales[i] - predictedSales[i]), 2)

# Calculate the RMSE by taking the square root of MSE 
RMSE = pow((sum/actualSales.size), 0.5)

# Print results
print('The RMSE between actual sales and predicted sales is {:.2f}'.format(RMSE))

The RMSE between actual sales and predicted sales is 1050.40


In [ ]:
# To identify which features are most associated with higher sales, I joined the OHE features matrix with the predicted sales, and then looked for high
# postive correlations

# Create a dataframe from predicted sales for correlation function consumption
predictedSales = pd.DataFrame(LR.predict(X))
predictedSales.rename(columns = {0:'Item_Outlet_Sales'}, inplace = True)

# Join the OHE features matrix and predicted sales
correlateMatrix = pd.concat([X, predictedSales], axis=1)

# Do the correlation
correlateMatrix.corr()

## Features associated with higher predicted sales. Suggestion is to emulate characteritics of each feature below across piers.

# 1) The single highest correlation is between Item_MRP and predicted sales
# 2) The best performing outlet in terms of predicted sales is OUT027
# 3) Medium sized outlets will far outperform both small and high outlets in terms of predicted sales
# 4) Supermarket Type 3 will by far be the best performing outlet type in terms of predicted sales    

,Item_Identifier_DRA12,Item_Identifier_DRA24,Item_Identifier_DRA59,Item_Identifier_DRB01,Item_Identifier_DRB13,Item_Identifier_DRB24,Item_Identifier_DRB25,Item_Identifier_DRB48,Item_Identifier_DRC01,Item_Identifier_DRC12,Item_Identifier_DRC13,Item_Identifier_DRC24,Item_Identifier_DRC25,Item_Identifier_DRC27,Item_Identifier_DRC36,Item_Identifier_DRC49,Item_Identifier_DRD01,Item_Identifier_DRD12,Item_Identifier_DRD13,Item_Identifier_DRD15,Item_Identifier_DRD24,Item_Identifier_DRD25,Item_Identifier_DRD27,Item_Identifier_DRD37,Item_Identifier_DRD49,Item_Identifier_DRD60,Item_Identifier_DRE01,Item_Identifier_DRE03,Item_Identifier_DRE12,Item_Identifier_DRE13,Item_Identifier_DRE15,Item_Identifier_DRE25,Item_Identifier_DRE27,Item_Identifier_DRE37,Item_Identifier_DRE48,Item_Identifier_DRE49,Item_Identifier_DRE60,Item_Identifier_DRF01,Item_Identifier_DRF03,Item_Identifier_DRF13,...,Item_Visibility,Item_Type_Soft Drinks,Item_Type_Dairy,Item_Type_Hard Drinks,Item_Type_Canned,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Snack Foods,Item_Type_Baking Goods,Item_Type_Starchy Foods,Item_Type_Meat,Item_Type_Seafood,Item_Type_Breakfast,Item_Type_Breads,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Others,Item_MRP,Outlet_Identifier_OUT010,Outlet_Identifier_OUT017,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT013,Outlet_Identifier_OUT018,Outlet_Identifier_OUT049,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT046,Outlet_Establishment_Year,Outlet_Size_Small,Outlet_Size_High,Outlet_Size_Medium,Outlet_Location_Type_Tier 3,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 1,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Outlet_Sales
Item_Identifier_DRA12,1.000000,-0.000761,-0.000814,-0.000498,-0.000643,-0.000575,-0.000704,-0.000761,-0.000704,-0.000575,-0.000575,-0.000407,-0.000761,-0.000704,-0.000643,-0.000498,-0.000643,-0.000761,-0.000761,-0.000761,-0.000761,-0.000814,-0.000643,-0.000761,-0.000575,-0.000704,-0.000407,-0.000761,-0.000704,-0.000643,-0.000704,-0.000643,-0.000704,-0.000643,-0.000761,-0.000863,-0.000704,-0.000814,-0.000814,-0.000704,...,-0.014421,0.113085,-0.007828,-0.004260,-0.007620,-0.008869,-0.010910,-0.010744,-0.007614,-0.003528,-0.006080,-0.002309,-0.003035,-0.004623,-0.006766,-0.009176,-0.003775,0.000372,0.010924,0.004948,0.004899,0.004911,0.004875,0.004924,-0.009289,-0.006821,-0.009317,-0.009289,0.010573,0.005550,0.004875,-0.009106,0.005812,0.009803,-0.016559,0.003156,0.000688,0.004924,-0.009317,-0.006434
Item_Identifier_DRA24,-0.000761,1.000000,-0.000879,-0.000538,-0.000695,-0.000621,-0.000761,-0.000822,-0.000761,-0.000621,-0.000621,-0.000439,-0.000822,-0.000761,-0.000695,-0.000538,-0.000695,-0.000822,-0.000822,-0.000822,-0.000822,-0.000879,-0.000695,-0.000822,-0.000621,-0.000761,-0.000439,-0.000822,-0.000761,-0.000695,-0.000761,-0.000695,-0.000761,-0.000695,-0.000822,-0.000932,-0.000761,-0.000879,-0.000879,-0.000761,...,-0.012438,0.122153,-0.008455,-0.004601,-0.008231,-0.009580,-0.011785,-0.011606,-0.008224,-0.003811,-0.006568,-0.002494,-0.003278,-0.004994,-0.007308,-0.009912,-0.004078,0.010633,0.009033,0.003152,0.003103,-0.010028,0.003078,-0.010022,0.003103,0.009622,0.003042,-0.010034,-0.009699,0.000490,0.003078,-0.002565,0.002085,-0.002509,0.000353,0.013657,-0.004999,-0.010022,0.003042,0.003477
Item_Identifier_DRA59,-0.000814,-0.000879,1.000000,-0.000575,-0.000743,-0.000664,-0.000814,-0.000879,-0.000814,-0.000664,-0.000664,-0.000470,-0.000879,-0.000814,-0.000743,-0.000575,-0.000743,-0.000879,-0.000879,-0.000879,-0.000879,-0.000940,-0.000743,-0.000879,-0.000664,-0.000814,-0.000470,-0.000879,-0.000814,-0.000743,-0.000814,-0.000743,-0.000814,-0.000743,-0.000879,-0.000997,-0.000814,-0.000940,-0.000940,-0.000814,...,0.050394,0.130594,-0.009040,-0.004919,-0.008800,-0.010242,-0.012600,-0.012408,-0.008793,-0.004075,-0.007022,-0.002666,-0.003505,-0.005339,-0.007813,-0.010597,-0.0